In [14]:
# ! pip install xmlschema
# ! pip install scikit-learn
# ! pip install openai
! pip install tiktoken
# ! pip install trl
# ! pip install lzma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
! python3 --version

Python 3.10.12


In [1]:

import os
import gc

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
# Print current working directory
# !ls -alh /var/model/Phind-CodeLlama-34B-v2
# Change to /var/model/Phind-CodeLlama-34B-v2
# os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
# Print current working directory
# os.getcwd()
! ls -alh /var/model/genie-in-the-box/src/ephemera/prompts/data

total 31M
drwxr--r-- 4 1001 1001 4.0K Dec 20 03:40 .
drwxr--r-- 4 1001 1001 4.0K Jan  2 16:18 ..
-rwxr--r-- 1 1001 1001 6.1K Jun 20  2023 .DS_Store
-rwxr--r-- 1 1001 1001 4.0K Jun 20  2023 ._.DS_Store
-rw-rw-r-- 1 1001 1001    0 Oct 13 18:48 __init__.py
drwxrwxr-x 2 1001 1001 4.0K Oct 13 18:48 fine-tuning-results
drwxrwxr-x 2 1001 1001 4.0K Oct 13 18:48 jsonl
-rw-rw-r-- 1 1001 1001 1.6K Oct 13 18:48 munger.py
-rw-rw-r-- 1 1001 1001  36K Dec 18 19:16 search-terms.txt
-rw-rw-r-- 1 1001 1001 7.0K Oct 13 18:48 synthetic-data-load-url-in-current-tab.txt
-rw-rw-r-- 1 1001 1001 7.3K Oct 13 18:48 synthetic-data-load-url-new-tab.txt
-rw-rw-r-- 1 1001 1001  13K Oct 13 18:48 synthetic-data-none-of-the-above.txt
-rw-rw-r-- 1 1001 1001 8.5K Dec 17 00:42 synthetic-data-search-google-in-current-tab.txt
-rw-rw-r-- 1 1001 1001  11K Oct 13 18:48 synthetic-data-search-google-in-new-tab.txt
-rw-rw-r-- 1 1001 1001 8.5K Oct 13 18:48 synthetic-data-search-google-scholar-in-current-tab.txt
-rw-rw-r-- 1 1001 1

In [3]:
import json
from xmlschema import XMLSchema

os.chdir( "/var/model/genie-in-the-box/src" )

import lib.utils.util     as du
import lib.utils.util_xml as dux
import lib.utils.util_pytorch as dupt

from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator


## Get Validation dataset

In [4]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl"
deepily_dataset_validation = du.get_file_as_list( path )
deepily_dataset_validation = [ json.loads( line ) for line in deepily_dataset_validation ]
len( deepily_dataset_validation )

1000

In [5]:
# Use the Task below and the Input given to write the Response, which is a programmatic instruction that can solve the following Task:
def prompt_instruction_format( sample ):
    
  return f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [6]:
for line in prompt_instruction_format( deepily_dataset_validation[ 0 ] ).split( "\n" ): print( line )

### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

        You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

        Requirement: You MUST NOT use python code to answer this question.
        Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
        Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
        Below is the raw human voice c

In [7]:
def generate_text( foo_tokenizer, model, question, max_new_tokens=256, debug=False ):
    
    instruction = f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

    You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

    Requirement: You MUST NOT use python code to answer this question.
    Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
    Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
    Below is the raw human voice command transcription formatted using simple XML:
    
    <human>
        <voice-command>{question}</voice-command>
    </human>
    
    The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
    
    <response>
        <browser-command></browser-command>
        <args></args>
    </response>

    Requirement: The first word of your response MUST be `<response>`

    ### Response:"""
    
    
    device = "cuda:0"
    inputs = foo_tokenizer( instruction, return_tensors="pt" ).to( device )
    
    stop_token_id = foo_tokenizer.encode( "</response>" )[ 0 ]
    
    generation_output = model.generate(
        input_ids=inputs[ "input_ids" ],
        attention_mask=inputs[ "attention_mask" ],
        max_new_tokens=max_new_tokens,
        eos_token_id=stop_token_id,
        pad_token_id=stop_token_id
    )
        
    if debug: 
        print( "generation_output[ 0 ]:", generation_output[ 0 ], end="\n\n" )
        print( "generation_output[ 0 ].shape:", generation_output[ 0 ].shape, end="\n\n" )
    
    # Skip decoding the prompt part of the output   
    input_length = inputs[ "input_ids" ].size( 1 )
    raw_output = foo_tokenizer.decode( generation_output[ 0 ][ input_length: ] )
    
    if debug: 
        print( "raw_output:", raw_output, end="\n\n" )
        print(  "len( raw_output ):", len( raw_output ), end="\n\n")
    
    # response   = raw_output.split( "### Response:" )[ 1 ]
    
    response = raw_output.replace( "</s><s>", "" ).strip()
    
    return response

question = "Ask Google scholar about QLORA and PEFT fine-tuning for XML output, show results in Another tab"

# for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

In [8]:
import os 
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )

In [9]:
# Drops 16.4/19.0 GB per GPU down to 3.25 GB per GPU!
import gc
base_model = None 
adapter_plus_model = None
gc.collect()
torch.cuda.empty_cache() 

2477

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
print( bnb_config )
tokenizer              = AutoTokenizer.from_pretrained( "." )
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

# Identify the second GPU
second_gpu = torch.device( 'cuda:1' ) if torch.cuda.is_available() else None

# ¡OJO! Why were we turning off the cash here? 
# We're not! It makes a huge performance difference: 21 vs 14 tokens per second!
base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="cuda:1", low_cpu_mem_usage=True, use_cache=True, use_flash_attention_2=True
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
dupt.print_device_allocation( base_model )


model.embed_tokens.weight: cuda:1
model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:1
model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:1
model.layers.0.self_attn.q_proj.base_layer.weight: cuda:1
model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:1
model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:1
model.layers.0.self_attn.k_proj.base_layer.weight: cuda:1
model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:1
model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:1
model.layers.0.self_attn.v_proj.base_layer.weight: cuda:1
model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:1
model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:1
model.layers.0.self_attn.o_proj.base_layer.weight: cuda:1
model.layers.0.mlp.gate_proj.lora_A.default.weight: cuda:1
model.layers.0.mlp.gate_proj.lora_B.default.weight: cuda:1
model.layers.0.mlp.gate_proj.base_layer.weight: cuda:1
model.layers.0.mlp.up_proj.lora_A.default.weight: cuda:1
model.la

In [12]:
response = generate_text( tokenizer, base_model, question )
response = f"[{response}]"
for line in response.split( "\n" ): print( line )

[<response>
            <browser-command>search google scholar new tab</browser-command>
            <args>QLORA and PEFT fine-tuning for XML output</args>
        </response>]


In [13]:
from peft import PeftModel
adapter_plus_model = PeftModel.from_pretrained( base_model, "adapters/00-browser-vox-command", use_flash_attention_2=True )

In [15]:
# Move the model to GPU
adapter_plus_model = adapter_plus_model.to( "cuda:1" ) # where device is your GPU device

In [16]:
dupt.print_device_allocation( adapter_plus_model )

base_model.model.model.embed_tokens.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight: cuda:1
base_model.model.

In [17]:
for line in generate_text( tokenizer, adapter_plus_model, question ).split( "\n" ): print( line )

<response>
            <browser-command>search google scholar new tab</browser-command>
            <args>QLORA and PEFT fine-tuning for XML output</args>
        </response>


In [20]:
import pandas as pd

In [25]:
validation_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100)
validation_df.head()

,instruction,input,output,prompt
584,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
268,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
164,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
594,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
64,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...


In [26]:
validation_df.shape

(100, 4)

In [27]:
questions = validation_df[ 'input' ].tolist()
for idx, question in enumerate( questions ):
    question = dux.get_value_by_xml_tag_name( question, "voice-command" )
    # print( question )
    questions[ idx ] = question
    
questions[ 0 ]

'Open a new tab and fetch Google Scholar results for What are common causes and solutions for errors related to incorrect syntax in Python?'

In [28]:
generate_text( tokenizer, adapter_plus_model, questions[ 0 ] )

'<response>\n            <browser-command>search google scholar new tab</browser-command>\n            <args>What are common causes and solutions for errors related to incorrect syntax in Python?</args>\n        </response>'

In [30]:
import re

In [31]:
responses = [ ]
for idx, question in enumerate( questions ):
    # Display progress every nth iteration
    print( f"[{idx + 1}] of [{len( questions )}] [{( ( idx + 1 ) / len( questions ) ) * 100:.1f}%] [{question}]" )
    response = generate_text( tokenizer, adapter_plus_model, question )
    response = re.sub( r'>\s+<', '><', response )
    print( f"[{response}]" )
    responses.append( response )
    

[1] of [100] [1.0%] [Open a new tab and fetch Google Scholar results for What are common causes and solutions for errors related to incorrect syntax in Python?]
[<response><browser-command>search google scholar new tab</browser-command><args>What are common causes and solutions for errors related to incorrect syntax in Python?</args></response>]
[2] of [100] [2.0%] [Start a Google search of Efficient file reading with Pandas in this tab]
[<response><browser-command>search google current tab</browser-command><args>Efficient file reading with Pandas</args></response>]
[3] of [100] [3.0%] [Search and provide results for Scikit-Learn documentation: Where can you find comprehensive documentation for Scikit-Learn? here]
[<response><browser-command>search current tab</browser-command><args>Scikit-Learn documentation: Where can you find comprehensive documentation for Scikit-Learn?</args></response>]
[4] of [100] [4.0%] [Delve for classic rock music in this window]
[<response><browser-command>

In [32]:
len( responses )

100

In [33]:
validation_df[ 'response' ] = responses
validation_df.head()

,instruction,input,output,prompt,response
584,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google schol...
268,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google curre...
164,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search current tab<...
594,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search current tab<...
64,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google schol...


In [34]:
# Install notebook magic that forces reload a source code
%load_ext autoreload

In [35]:
os.chdir( "/var/model/genie-in-the-box/src" )
os.getcwd()

'/var/model/genie-in-the-box/src'

In [36]:
%autoreload
from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator

# # Advice from phind when auto reload fails: https://www.phind.com/search?cache=l7c51x1v3tids6l43mgkkm9j
# os.chdir( "/var/model/genie-in-the-box/src" )
# %run "ephemera/prompts/xml_fine_tuning_prompt_generator.py"  

xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box" )

validation_df = xml_ftp_generator.validate_prompts_and_responses( validation_df )

xml_ftp_generator.print_validation_stats( validation_df )

#  Results from a 1000 sample run:
# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 99.3%
# Response has correct values 99.3%
#  Browser command is correct 99.6%
#             Args is correct 99.7%

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True
------------------------------------------------------------------------------------------------------------------------
- Validation Stats
------------------------------------------------------------------------------------------------------------------------

               Is valid xml 100.0%
          Contains response 100.0%
   Contains browser command 100.0%
              Contains args 100.0%
          Response is exact 98.0%
Response has correct values 98.0%
 Browser command is correct 99.0%
            Args is correct 99.0%
